<a href="https://colab.research.google.com/github/SeanSDarcy2001/CISProgrammingAssignments/blob/main/CIS1HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Integrated Surgery: Programming Assignment 1

## PART 1
Develop (or develop proficiency with) a Cartesian math package for 3D points, rotations, and frame transformations.

In [ ]:
## use package to represent 3D points
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

p1    = np.array([-1489., -4913.,  4345.])
p2    = np.array([ 2633., -3268.,  5249.])
pcalc = np.array([-3210., -4390.,  3930.])

def norm(v):
    return v / np.sqrt(np.dot(v, v))

In [ ]:
## use packages to represent 3D rotations
def identity():
  return np.array([[1, 0,  0], [0, 1, 0], [0, 0, 1]])

# generate skew matrix from 1x3 array 
def skew(v) :
  skewArg = norm(v)
  ax = skewArg[0]
  ay = skewArg[1]
  az = skewArg[2]
  skew = np.array([[0,    -az,  ay],
                   [az,   0,    -ax],
                   [-ay,  ax,   0]])
  return skew

# generate 3D Rotation matrix
# @Params: alpha, beta, gamma - Rotation angles about the x, y, and z axes respectively
# @Returns: 3D Rotation matrix
def RotfromAngles(alpha, beta, gamma) :
  Rx = np.array([[1, 0,  0], [0, np.cos(alpha), -np.sin(alpha)], [0, np.sin(alpha), np.cos(alpha)]])
  Ry = np.array([[np.cos(beta), 0,  np.sin(beta)], [0, 1, 0], [-np.sin(beta), 0, np.cos(beta)]])
  Rz = np.array([[np.cos(gamma), -np.sin(gamma),  0], [np.sin(gamma), np.cos(gamma), 0], [0, 0, 1]])
  R = RotfromComponents(Rx, Ry, Rz)
  return R

def smallAngleR(v, theta) :
  n = norm(v)
  return (identity() + skew(theta * n))

def RotfromComponents(Rx, Ry, Rz) :
  R = np.dot(np.dot(Rx, Ry), Rz)
  return R

# convert 3D vector into 4D homogeneous coords
def homogenousVector(v, scale) :
  Vx = scale * v[0]
  Vy = scale * v[1]
  Vz = scale * v[2]
  return np.array([Vx, Vy, Vz, scale])

R = RotfromAngles(0, 1, 0)
print(R)

# theta = int
# RxTheta = np.array([1, 0,  0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]))
# RyTheta

[[ 0.54030231  0.          0.84147098]
 [ 0.          1.          0.        ]
 [-0.84147098  0.          0.54030231]]


In [ ]:
## use packages to represent 3D frame transformations

class Frame:
  
  def __init__(self, R, p):
    self.R = R
    self.p = p
    self.F = self.getFrame()

  # define Frame using Rotation and Translation
  # @Params: Rotation R and Translation p
  # @Return: Frame
  def getFrame(self):
    F = np.row_stack((np.column_stack((self.R, self.p)),
                      [0,0,0,1]))
    return F

  # applies frame transformation to vector
  # @Params: Frame F and Vector v
  # @Return: transformed vector
  def appFrame(self, v):
    return np.dot(self.F, v)

  # applies inverse frame transformation to vector
  # @Params: Frame F and Vector v
  # @Return: transformed vector
  def appInvFrame(self, v):
    Finv = Frame(self.R.T, - (np.dot(self.R.T, self.p)))
    return np.dot(Finv.getFrame(), v)

# test cases
p = np.array([5, 7, 9])
F = Frame(R, p)
print(F.getFrame())

one_t = np.array([[0,0,0],[1,0,0],[0,1,0]])
two_t = np.array([[0,0,1], [1,0,1],[0,0,2]])
# Transformation is 90 degree rot over x and +1 translation on z
# Tested here
R = RotfromAngles(90, 0, 0)
p = (0,0,1)
F = Frame(R, p)
n = homogenousVector(one_t[0],1)
print((n))
print(F.appFrame(n))

[[ 0.54030231  0.          0.84147098  5.        ]
 [ 0.          1.          0.          7.        ]
 [-0.84147098  0.          0.54030231  9.        ]
 [ 0.          0.          0.          1.        ]]
[0 0 0 1]
[0. 0. 1. 1.]


In [ ]:
# ## example?
# n1, n2, ncalc = [norm(p) for p in [p1, p2, pcalc]]

# new_zaxis  = norm(np.cross(n1, n2))
# new_xaxis  = n1
# zero       = np.zeros(3) 

# fig = plt.figure(figsize=[10, 8])
# ax  = fig.add_subplot(1, 1, 1, projection='3d')

# x, y, z = zip(zero, new_xaxis)
# plt.plot(x, y, z, '-k', linewidth=3)

# x, y, z = zip(zero, new_zaxis)
# plt.plot(x, y, z, '--k', linewidth=3)

# x, y, z = zip(zero, n2)
# plt.plot(x, y, z, '-r', linewidth=1)

# x, y, z = zip(zero, ncalc)
# plt.plot(x, y, z, '--g', linewidth=1)

# ax.set_xlim(-1, 1)
# ax.set_ylim(-1, 1)
# ax.set_zlim(-1, 1)

# plt.show()

## PART 2

Develop a 3D point set to 3D point set registration algorithm

K. Arun, et. al., IEEE PAMI, Vol 9, no 5, pp 698-700, Sept 1987 

In [ ]:
def rigid_registration(a, b):
    # Calculate and subtract means
    a_c = np.expand_dims(np.mean(a, axis = 1), axis=1)
    b_c = np.expand_dims(np.mean(b, axis = 1), axis=1)
    A = a - a_c
    B = b - b_c

    # Compute SVD
    H = np.dot(A, B.T)
    U, X, V_t = np.linalg.svd(H)
    R = np.dot(V_t.T, U.T)
    assert np.allclose(np.linalg.det(R), 1)
    p = b_c - np.matmul(R, a_c)
    return Frame(R, p)

In [ ]:
# Two sets of points to check 3D-3D registration
one_t = np.array([[0,0,0],[1,0,0],[0,1,0]])
two_t = np.array([[0,0,1], [1,0,1],[0,0,2]])

reg = rigid_registration(one_t.T, two_t.T)
res = np.dot(reg.R, one_t.T) + reg.p
if np.allclose(res, two_t.T):
    print("Success!")
else:
    print("Registration failed.")

Success!


## PART 3

Develop a “pivot” calibration method

# Contributions

## Seby


*   developed proficiency with a Cartesian math package for 3D points rotations, and frame transformations. (1)


## Alex


*   developed proficiency with a Cartesian math package for 3D points rotations, and frame transformations. (1)
*   created Frame object with transformation functionality. (1)
*   developed algorithm for 3D-3D registration. (2)

